In [47]:
from langchain.tools.retriever import create_retriever_tool
from langchain_community.vectorstores import FAISS
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain_openai import ChatOpenAI, OpenAIEmbeddings
from langchain_core.messages import AIMessage, HumanMessage
from langchain_community.document_loaders import UnstructuredXMLLoader
from langchain import hub
from langchain.agents import AgentExecutor, create_openai_tools_agent
from langchain_community.document_loaders import DirectoryLoader
from langchain.document_loaders import DirectoryLoader, TextLoader
from sentence_transformers import SentenceTransformer
from langchain_community.embeddings import HuggingFaceEmbeddings

In [48]:
loader = DirectoryLoader(
    "./data", glob="**/*.json", show_progress=True, loader_cls=TextLoader
)
docs = loader.load()

100%|██████████| 38275/38275 [00:06<00:00, 6309.98it/s]


In [49]:
text_splitter = RecursiveCharacterTextSplitter(chunk_size=10_000, chunk_overlap=150)
docs = text_splitter.split_documents(documents=docs)

In [50]:
embeddings = HuggingFaceEmbeddings(
    model_name="all-MiniLM-L6-v2", model_kwargs={"device": "mps"}
)
vectorstore = FAISS.from_documents(
    docs,
    embedding=embeddings,
)
vectorstore.save_local("faiss_index")
retriever = vectorstore.as_retriever()